## [0] 실습 소개

이번 실습에서 해볼 내용
- Context-Aware Recommender System (CARS)
  - FM (Factorization Machine)
  - FFM (Field-Aware Factorization Machine)
- Deep Learning Recommender System
  - NCF (Neural Collaborative Filtering)
  - AutoRec
  - WDN (Wide & Deep Network)
  - DCN (Deep & Cross Network)

RecSys 기초 대회 강의에서는 Book Crossing 데이터를 사용하여, 모든 실습 및 미션, 대회를 진행합니다. [Kaggle Book-Crossing](https://www.kaggle.com/datasets/ruchi798/bookcrossing-dataset) 을 출처로 하며, 데이터는 재구성되어 제공되었습니다. 해당 데이터는 CC0: Public Domain 라이센스임을 밝힙니다.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset
from torch.autograd import Variable
import numpy as np
import math
import tqdm
import pdb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import warnings

warnings.filterwarnings(action='ignore')

# [1] 데이터 불러오기
book data를 이용합니다. 4강 실습용으로 샘플링된 데이터를 이용하며, 아래 코드를 통해 불러올 수 있습니다.

In [ ]:
!wget --load-cookies ~/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies ~/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1jdLFf4JyfWo1406LJ2f8no67YPf15X3f' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1jdLFf4JyfWo1406LJ2f8no67YPf15X3f" -O users.csv && rm -rf ~/cookies.txt
!wget --load-cookies ~/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies ~/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-EZ2fFCA5RNoqlyM69NeN-L4Y6qTLnQN' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-EZ2fFCA5RNoqlyM69NeN-L4Y6qTLnQN" -O books.csv && rm -rf ~/cookies.txt
!wget --load-cookies ~/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies ~/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-I3YKtaJb5IPvOOFqkonQj5ikJQHfoUC' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1-I3YKtaJb5IPvOOFqkonQj5ikJQHfoUC" -O ratings.csv && rm -rf ~/cookies.txt

--2022-10-12 14:49:31--  https://docs.google.com/uc?export=download&confirm=&id=1jdLFf4JyfWo1406LJ2f8no67YPf15X3f
Resolving docs.google.com (docs.google.com)... 142.251.31.113, 142.251.31.138, 142.251.31.101, ...
Connecting to docs.google.com (docs.google.com)|142.251.31.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-00-94-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ls1nnfsjjjfitbnm8f98qok0trmnrcsi/1665586125000/15943349948421319767/*/1jdLFf4JyfWo1406LJ2f8no67YPf15X3f?e=download&uuid=a32042a3-947f-4255-848f-ff4ce906bf06 [following]
--2022-10-12 14:49:32--  https://doc-00-94-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/ls1nnfsjjjfitbnm8f98qok0trmnrcsi/1665586125000/15943349948421319767/*/1jdLFf4JyfWo1406LJ2f8no67YPf15X3f?e=download&uuid=a32042a3-947f-4255-848f-ff4ce906bf06
Resolving doc-00-94-docs.googleusercontent.com (doc-00-94-docs.googleusercontent.com)... 74.125.143.132, 2

In [ ]:
books = pd.read_csv('./books.csv')
users = pd.read_csv('./users.csv')
ratings = pd.read_csv('./ratings.csv')

해당 데이터셋에서는 isbn을 item_id로 볼 수 있으며, user_id가 users_id로 볼 수 있습니다.

implicit 형태로 0/1 문제로 변경하여 진행합니다

In [ ]:
ratings['rating'] = [int(i/7) for i in ratings['rating']]

하이퍼 파라미터를 포함한 미리 정의할 내용들을 미리 선언하겠습니다

In [ ]:
######## Hyperparameter ########
batch_size = 256 # 배치 사이즈
data_shuffle = True
embed_dim = 8 # embed feature의 dimension
epochs = 100 # epoch 돌릴 횟수
learning_rate = 0.001 # 학습이 반영되는 정도를 나타내는 파라미터
weight_decay=1e-6 # 정규화를 위한 파라미터
gpu_idx = 0

seed=42

torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

device = torch.device("cuda:{}".format(gpu_idx) if torch.cuda.is_available() else "cpu")
print(device)

cpu


## train, test 데이터로 split 해줍니다

FM/FFM에 필요한 데이터셋

In [ ]:
def age_map(x: int) -> int:
    x = int(x)
    if x < 20:
        return 1
    elif x >= 20 and x < 30:
        return 2
    elif x >= 30 and x < 40:
        return 3
    elif x >= 40 and x < 50:
        return 4
    elif x >= 50 and x < 60:
        return 5
    else:
        return 6

user_id와 isbn 정보에 대해 값을 직접 사용하는 대신 인덱스화 하여 인덱스된 값을 사용합니다

In [ ]:
user2idx = {v:k for k,v in enumerate(ratings['user_id'].unique())}
book2idx = {v:k for k,v in enumerate(ratings['isbn'].unique())}

In [ ]:
users_context = users.copy()
books_context = books.copy()

# 인덱싱 처리된 데이터 조인
context_df = ratings.merge(users_context, on='user_id', how='inner').merge(books_context[['isbn', 'category', 'publisher', 'language']], on='isbn', how='inner')

# 인덱싱 처리합니다
context_df['user_id'] = context_df['user_id'].map(user2idx)
context_df['isbn'] = context_df['isbn'].map(book2idx)

# user 파트 인덱싱
loc_city2idx = {v:k for k,v in enumerate(context_df['location_city'].unique())}
loc_state2idx = {v:k for k,v in enumerate(context_df['location_state'].unique())}
loc_country2idx = {v:k for k,v in enumerate(context_df['location_country'].unique())}
context_df['location_city'] = context_df['location_city'].map(loc_city2idx)
context_df['location_state'] = context_df['location_state'].map(loc_state2idx)
context_df['location_country'] = context_df['location_country'].map(loc_country2idx)
context_df['age'] = context_df['age'].apply(age_map)

# book 파트 인덱싱
category2idx = {v:k for k,v in enumerate(context_df['category'].unique())}
publisher2idx = {v:k for k,v in enumerate(context_df['publisher'].unique())}
language2idx = {v:k for k,v in enumerate(context_df['language'].unique())}
context_df['category'] = context_df['category'].map(category2idx)
context_df['publisher'] = context_df['publisher'].map(publisher2idx)
context_df['language'] = context_df['language'].map(language2idx)

context_df

,user_id,isbn,rating,age,location_country,location_state,location_city,category,publisher,language
0,0,0,1,2,0,0,0,0,0,0
1,61,0,1,4,0,1,1,0,0,0
2,76,0,1,3,0,2,2,0,0,0
3,103,0,1,3,0,3,3,0,0,0
4,130,0,0,3,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
16760,246,12507,0,3,0,11,135,377,39,0
16761,246,12508,1,3,0,11,135,0,643,0
16762,246,12509,0,3,0,11,135,4,560,0
16763,246,12510,1,3,0,11,135,17,730,0


In [ ]:
# fm/ffm에 사용할 데이터셋
X_train_context, X_test_context, y_train_context, y_test_context = train_test_split(
  context_df.drop(['rating'], axis=1), context_df['rating'], test_size=0.2, random_state=seed)

# PyTorch의 DataLoader에서 사용할 수 있도록 변환 
train_dataset_context = TensorDataset(torch.IntTensor(X_train_context.values), torch.IntTensor(y_train_context.values))
test_dataset_context = TensorDataset(torch.IntTensor(X_test_context.values), torch.IntTensor(y_test_context.values))

# DataLoader로 변환
train_dataloader_context = DataLoader(train_dataset_context, batch_size=batch_size, shuffle=data_shuffle)
test_dataloader_context = DataLoader(test_dataset_context, batch_size=batch_size, shuffle=False)

In [ ]:
# user_id, isbn, rating만 있는 데이터
ratings['user_id'] = ratings['user_id'].map(user2idx)
ratings['isbn'] = ratings['isbn'].map(book2idx)

r_train_X, r_test_X, r_train_y, r_test_y = train_test_split(
    ratings[['user_id', 'isbn']], ratings['rating'], test_size=0.2, random_state=seed)
print('학습 데이터 크기:', r_train_X.shape, r_train_y.shape)
print('테스트 데이터 크기:', r_test_X.shape, r_test_y.shape)

# PyTorch의 DataLoader에서 사용할 수 있도록 변환 
r_train_dataset = TensorDataset(torch.IntTensor(r_train_X.values), torch.IntTensor(r_train_y.values))
r_test_dataset = TensorDataset(torch.IntTensor(r_test_X.values), torch.IntTensor(r_test_y.values))

# DataLoader로 변환
r_train_dataloader = DataLoader(r_train_dataset, batch_size=batch_size, shuffle=data_shuffle)
r_test_dataloader = DataLoader(r_test_dataset, batch_size=batch_size, shuffle=False)

학습 데이터 크기: (13412, 2) (13412,)
테스트 데이터 크기: (3353, 2) (3353,)


In [ ]:
# autoencoder에 사용할 X, y 구분 없이 나누는 데이터
ae_train, ae_test = train_test_split(
    ratings[['user_id', 'isbn', 'rating']], test_size=0.2, random_state=seed)

print('학습 데이터 크기:', ae_train.shape)
print('테스트 데이터 크기:', ae_test.shape)

학습 데이터 크기: (13412, 3)
테스트 데이터 크기: (3353, 3)


# [2] 학습 함수

본 실습에서는 rmse를 사용하지 않지만 아래 코드를 활용하여 필요시 사용하시면 되겠습니다

In [ ]:
from sklearn.metrics import mean_squared_error
def rmse(true: list, pred: list) -> float:
    return np.sqrt(mean_squared_error(true, pred))

In [ ]:
def train(model: type, optimizer: torch.optim, data_loader: DataLoader, criterion: torch.nn, device: str, log_interval: int=100) -> None:
    model.train()
    total_loss = 0
    tk0 = tqdm.tqdm(data_loader, smoothing=0, mininterval=1.0)
    for i, (fields, target) in enumerate(tk0):
        fields, target = fields.to(device), target.to(device)
        y = model(fields)
        loss = criterion(y, target.float())
        model.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        if (i + 1) % log_interval == 0:
            tk0.set_postfix(loss=total_loss / log_interval)
            total_loss = 0

def test(model: type, data_loader: DataLoader, device: str) -> float:
    model.eval()
    targets, predicts = list(), list()
    with torch.no_grad():
        for fields, target in tqdm.tqdm(data_loader, smoothing=0, mininterval=1.0):
            fields, target = fields.to(device), target.to(device)
            y = model(fields)
            targets.extend(target.tolist())
            predicts.extend(y.tolist())
    return roc_auc_score(targets, predicts)

# [3] Context-Aware Recommender System (CARS)

In [ ]:
class FactorizationMachine(nn.Module):

    def __init__(self, reduce_sum:bool=True):
        super().__init__()
        self.reduce_sum = reduce_sum

    def forward(self, x: torch.Tensor):
        """
        :param x: Float tensor of size ``(batch_size, num_fields, embed_dim)``
        """
        square_of_sum = torch.sum(x, dim=1) ** 2
        sum_of_square = torch.sum(x ** 2, dim=1)
        ix = square_of_sum - sum_of_square
        if self.reduce_sum:
            ix = torch.sum(ix, dim=1, keepdim=True)
        return 0.5 * ix

In [ ]:
class FeaturesEmbedding(nn.Module):

    def __init__(self, field_dims: np.ndarray, embed_dim: int):
        super().__init__()
        self.embedding = torch.nn.Embedding(sum(field_dims), embed_dim)
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.long)
        torch.nn.init.xavier_uniform_(self.embedding.weight.data)

    def forward(self, x: torch.Tensor):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        x = x + x.new_tensor(self.offsets).unsqueeze(0)
        return self.embedding(x)

In [ ]:
class FeaturesLinear(nn.Module):

    def __init__(self, field_dims: np.ndarray, output_dim: int=1):
        super().__init__()
        self.fc = torch.nn.Embedding(sum(field_dims), output_dim)
        self.bias = torch.nn.Parameter(torch.zeros((output_dim,)))
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.long)

    def forward(self, x: torch.Tensor):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        x = x + x.new_tensor(self.offsets).unsqueeze(0)
        return torch.sum(self.fc(x), dim=1) + self.bias

## FM

In [ ]:
class FactorizationMachineModel(nn.Module):

    def __init__(self, field_dims: np.ndarray, embed_dim: int):
        super().__init__()
        self.embedding = FeaturesEmbedding(field_dims, embed_dim)
        self.linear = FeaturesLinear(field_dims)
        self.fm = FactorizationMachine(reduce_sum=True)

    def forward(self, x: torch.Tensor):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        x = self.linear(x) + self.fm(self.embedding(x))
        return torch.sigmoid(x.squeeze(1))

In [ ]:
fm_field_dims = np.array([len(user2idx), len(book2idx), 6, len(loc_city2idx), len(loc_state2idx), len(loc_country2idx), 
                          len(category2idx), len(publisher2idx), len(language2idx)], dtype=np.uint32)

In [ ]:
criterion = torch.nn.BCELoss()
model = FactorizationMachineModel(fm_field_dims, embed_dim).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate, amsgrad=True, weight_decay=weight_decay)

for epoch in range(epochs):
    train(model, optimizer, train_dataloader_context, criterion, device)
    auc_score = test(model, train_dataloader_context, device)
    print('epoch:', epoch, 'validation: roc_auc_score:', auc_score)

100%|██████████| 53/53 [00:00<00:00, 301.77it/s]


epoch: 0 validation: roc_auc_score: 0.5129181284386842


100%|██████████| 53/53 [00:00<00:00, 549.01it/s]


epoch: 1 validation: roc_auc_score: 0.6025077477322605


100%|██████████| 53/53 [00:00<00:00, 532.90it/s]


...

epoch: 97 validation: roc_auc_score: 0.9968108775277401


100%|██████████| 53/53 [00:00<00:00, 572.41it/s]


epoch: 98 validation: roc_auc_score: 0.9968634307300974


100%|██████████| 53/53 [00:00<00:00, 536.02it/s]

epoch: 99 validation: roc_auc_score: 0.9968954300574162


In [ ]:
test(model, test_dataloader_context, device)

100%|██████████| 14/14 [00:00<00:00, 143.90it/s]


0.7134581442838162

## FFM

In [ ]:
class FieldAwareFactorizationMachine(nn.Module):

    def __init__(self, field_dims: np.ndarray, embed_dim: int):
        super().__init__()
        self.num_fields = len(field_dims)
        self.embeddings = torch.nn.ModuleList([
            torch.nn.Embedding(sum(field_dims), embed_dim) for _ in range(self.num_fields)
        ])
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.long)
        for embedding in self.embeddings:
            torch.nn.init.xavier_uniform_(embedding.weight.data)

    def forward(self, x: torch.Tensor):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        x = x + x.new_tensor(self.offsets).unsqueeze(0)
        xs = [self.embeddings[i](x) for i in range(self.num_fields)]
        ix = list()
        for i in range(self.num_fields - 1):
            for j in range(i + 1, self.num_fields):
                ix.append(xs[j][:, i] * xs[i][:, j])
        ix = torch.stack(ix, dim=1)
        return ix

In [ ]:
class FieldAwareFactorizationMachineModel(nn.Module):

    def __init__(self, field_dims: np.ndarray, embed_dim: int):
        super().__init__()
        self.linear = FeaturesLinear(field_dims)
        self.ffm = FieldAwareFactorizationMachine(field_dims, embed_dim)

    def forward(self, x: torch.Tensor):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        ffm_term = torch.sum(torch.sum(self.ffm(x), dim=1), dim=1, keepdim=True)
        x = self.linear(x) + ffm_term
        return torch.sigmoid(x.squeeze(1))

각 필드의 아이템 수만큼 field_dims 선언해주겠습니다

In [ ]:
ffm_field_dims = np.array([len(user2idx), len(book2idx), 6, len(loc_country2idx), len(loc_state2idx), len(loc_city2idx), 
                      len(category2idx), len(publisher2idx), len(language2idx)], 
                      dtype=np.uint32)

In [ ]:
criterion = torch.nn.BCELoss()
model = FieldAwareFactorizationMachineModel(ffm_field_dims, embed_dim).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate, amsgrad=True, weight_decay=weight_decay)

for epoch in range(epochs):
    train(model, optimizer, train_dataloader_context, criterion, device)
    auc_score = test(model, train_dataloader_context, device)
    print('epoch:', epoch, 'validation: roc_auc_score:', auc_score)

100%|██████████| 53/53 [00:00<00:00, 330.43it/s]


epoch: 0 validation: roc_auc_score: 0.557266163956835


100%|██████████| 53/53 [00:00<00:00, 345.55it/s]


epoch: 1 validation: roc_auc_score: 0.6594477822153109


100%|██████████| 53/53 [00:00<00:00, 335.79it/s]


...

epoch: 97 validation: roc_auc_score: 1.0


100%|██████████| 53/53 [00:00<00:00, 228.63it/s]


epoch: 98 validation: roc_auc_score: 1.0


100%|██████████| 53/53 [00:00<00:00, 315.13it/s]

epoch: 99 validation: roc_auc_score: 1.0


In [ ]:
test(model, test_dataloader_context, device)

100%|██████████| 14/14 [00:00<00:00, 300.55it/s]


0.7365658121934642

# [4] Deep Learning Recommender System

## NCF

In [ ]:
class MultiLayerPerceptron(nn.Module):

    def __init__(self, input_dim, embed_dims, dropout, output_layer=True):
        super().__init__()
        layers = list()
        for embed_dim in embed_dims:
            layers.append(torch.nn.Linear(input_dim, embed_dim))
            layers.append(torch.nn.BatchNorm1d(embed_dim))
            layers.append(torch.nn.ReLU())
            layers.append(torch.nn.Dropout(p=dropout))
            input_dim = embed_dim
        if output_layer:
            layers.append(torch.nn.Linear(input_dim, 1))
        self.mlp = torch.nn.Sequential(*layers)

    def forward(self, x):
        """
        :param x: Float tensor of size ``(batch_size, embed_dim)``
        """
        return self.mlp(x)

In [ ]:
class NeuralCollaborativeFiltering(nn.Module):

    def __init__(self, field_dims, user_field_idx, item_field_idx, embed_dim, mlp_dims, dropout):
        super().__init__()
        self.user_field_idx = user_field_idx
        self.item_field_idx = item_field_idx
        self.embedding = FeaturesEmbedding(field_dims, embed_dim)
        self.embed_output_dim = len(field_dims) * embed_dim
        self.mlp = MultiLayerPerceptron(self.embed_output_dim, mlp_dims, dropout, output_layer=False)
        self.fc = torch.nn.Linear(mlp_dims[-1] + embed_dim, 1)

    def forward(self, x):
        """
        :param x: Long tensor of size ``(batch_size, num_user_fields)``
        """
        x = self.embedding(x)
        user_x = x[:, self.user_field_idx].squeeze(1)
        item_x = x[:, self.item_field_idx].squeeze(1)
        gmf = user_x * item_x
        x = self.mlp(x.view(-1, self.embed_output_dim))
        x = torch.cat([gmf, x], dim=1)
        x = self.fc(x).squeeze(1)
        return torch.sigmoid(x)

In [ ]:
field_dims = np.array([len(user2idx), len(book2idx)], dtype=np.uint32)

In [ ]:
criterion = torch.nn.BCELoss()
model = NeuralCollaborativeFiltering(field_dims=field_dims, 
                                    user_field_idx=np.array((0, ), dtype=np.long), item_field_idx=np.array((1, ), dtype=np.long), 
                                    embed_dim=embed_dim, mlp_dims=(16, 16), dropout=0.2).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate, amsgrad=True, weight_decay=weight_decay)

for epoch in range(epochs):
    train(model, optimizer, r_train_dataloader, criterion, device)
    auc_score = test(model, r_train_dataloader, device)
    print('epoch:', epoch, 'validation: roc_auc_score:', auc_score)

100%|██████████| 53/53 [00:00<00:00, 540.15it/s]


epoch: 0 validation: roc_auc_score: 0.8140286883653547


100%|██████████| 53/53 [00:00<00:00, 541.48it/s]


epoch: 1 validation: roc_auc_score: 0.9395223867979836


100%|██████████| 53/53 [00:00<00:00, 519.93it/s]


...

epoch: 97 validation: roc_auc_score: 0.999997475487707


100%|██████████| 53/53 [00:00<00:00, 572.42it/s]


epoch: 98 validation: roc_auc_score: 0.9999977399604232


100%|██████████| 53/53 [00:00<00:00, 541.93it/s]

epoch: 99 validation: roc_auc_score: 0.999998028476114


In [ ]:
test(model, r_test_dataloader, device)

100%|██████████| 14/14 [00:00<00:00, 485.36it/s]


0.7097445971603072

## AutoRec

In [ ]:
# AutoRec에 맞는 형태로 데이터 변형해주기 위한 클래스
class RecDataset(Dataset):
    def __init__(self, n_user, n_item, list_user, list_item, data):
        self.n_user = n_user
        self.n_item = n_item
        self.list_user = list_user
        self.list_item = list_item
        
        rating = torch.tensor(data.rating.values)
        user_id = data.user_id
        item_id = data.isbn
        indices = torch.tensor(list(zip(user_id, item_id))).t()

        self.X = (
            torch.sparse_coo_tensor(indices, rating, (n_user, n_item))
            .to_dense()
            .to(dtype=torch.float)
        )

        ones = torch.ones_like(rating)
        self.mask = (
            torch.sparse_coo_tensor(indices, ones, (n_user, n_item))
            .to_dense()
            .to(dtype=torch.float)
        )


    def __getitem__(self, index):
        return self.X[index], self.mask[index]

    def __len__(self):
        return self.n_user

    def get_mat(self):
        return self.X, self.mask

In [ ]:
class AutoEncoder(nn.Module):
    def __init__(self, d, k, dropout=0.1):
        super(AutoEncoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(d, k),
            nn.Dropout(dropout),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(k, d),
            nn.Dropout(dropout),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.decoder(self.encoder(x))

In [ ]:
class AutoRec(nn.Module):
    def __init__(self, d, k, learning_rate, batch_size):
        super(AutoRec, self).__init__()
        self.batch_size = batch_size
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model = AutoEncoder(d, k).to(self.device)
        self.optimizer = torch.optim.SGD(self.model.parameters(), learning_rate, momentum=0.9, weight_decay=1e-4)
        self.feature_size = d # n_user/n_item

    def run(self, trainset, testlist, num_epoch):
        for epoch in range(1, num_epoch + 1):
            train_loader = DataLoader(trainset, self.batch_size, shuffle=True, pin_memory=True)
            self.train(train_loader, epoch)
            self.test(trainset, testlist)
            
    def train(self, train_loader, epoch):
        self.model.train()
        features = Variable(torch.FloatTensor(self.batch_size, self.feature_size))
        masks = Variable(torch.FloatTensor(self.batch_size, self.feature_size))

        for bid, (feature, mask) in enumerate(train_loader):
            if mask.shape[0] == self.batch_size:
                features.data.copy_(feature)
                masks.data.copy_(mask)
            else:
                features = Variable(feature)
                masks = Variable(mask)
            features, masks = features.to(self.device), masks.to(self.device)
            self.optimizer.zero_grad()
            output = self.model(features)
            loss = F.mse_loss(output * masks, features * masks)
            loss.backward()
            self.optimizer.step()

        print ("Epoch %d, train end." % epoch)

    def test(self, trainset, testlist):
        self.model.eval()
        with torch.no_grad():
            x_mat, mask = trainset.get_mat()
            features = Variable(x_mat).to(self.device)
            xc = self.model(features)
            xc = xc.cpu().data.numpy()

            rmse = 0.0
            for idx, (i, j, r) in testlist.iterrows():
                i, j = int(i), int(j)
                rmse += (xc[i][j]-r)*(xc[i][j]-r)
            rmse = math.sqrt(rmse / len(testlist))

        print (" Test RMSE = %f" % rmse)

In [ ]:
train_set = RecDataset(len(user2idx), len(book2idx), ratings['user_id'].unique(), ratings['isbn'].unique(), ae_train)

In [ ]:
d = len(book2idx)
mod = AutoRec(d=d, k=embed_dim,
          learning_rate = learning_rate,
          batch_size=batch_size)

mod.run(train_set, ae_test, num_epoch=epochs)

Epoch 1, train end.
 Test RMSE = 0.504176
Epoch 2, train end.
 Test RMSE = 0.504176
Epoch 3, train end.
 Test RMSE = 0.504176
Epoch 4, train end.
 Test RMSE = 0.504176
Epoch 5, train end.
 Test RMSE = 0.504176
Epoch 6, train end.
 Test RMSE = 0.504176
Epoch 7, train end.
 Test RMSE = 0.504176
Epoch 8, train end.
 Test RMSE = 0.504176
Epoch 9, train end.
 Test RMSE = 0.504176
Epoch 10, train end.
 Test RMSE = 0.504176
Epoch 11, train end.
 Test RMSE = 0.504176
Epoch 12, train end.
 Test RMSE = 0.504176
Epoch 13, train end.
 Test RMSE = 0.504176
Epoch 14, train end.
 Test RMSE = 0.504176
Epoch 15, train end.
 Test RMSE = 0.504176
Epoch 16, train end.
 Test RMSE = 0.504176
Epoch 17, train end.
 Test RMSE = 0.504176
Epoch 18, train end.
 Test RMSE = 0.504176
Epoch 19, train end.
 Test RMSE = 0.504176
Epoch 20, train end.
 Test RMSE = 0.504176
Epoch 21, train end.
 Test RMSE = 0.504176
Epoch 22, train end.
 Test RMSE = 0.504176
Epoch 23, train end.
 Test RMSE = 0.504176
Epoch 24, train end.

## WDN

In [ ]:
class WideAndDeepModel(nn.Module):

    def __init__(self, field_dims: np.ndarray, embed_dim: int, mlp_dims: tuple, dropout: float):
        super().__init__()
        self.linear = FeaturesLinear(field_dims)
        self.embedding = FeaturesEmbedding(field_dims, embed_dim)
        self.embed_output_dim = len(field_dims) * embed_dim
        self.mlp = MultiLayerPerceptron(self.embed_output_dim, mlp_dims, dropout)

    def forward(self, x: torch.Tensor):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        embed_x = self.embedding(x)
        x = self.linear(x) + self.mlp(embed_x.view(-1, self.embed_output_dim))
        return torch.sigmoid(x.squeeze(1))

In [ ]:
field_dims = np.array([len(user2idx), len(book2idx)], dtype=np.uint32)

In [ ]:
criterion = torch.nn.BCELoss()
model = WideAndDeepModel(field_dims, embed_dim, mlp_dims=(16, 16), dropout=0.2).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate, amsgrad=True, weight_decay=weight_decay)

for epoch in range(epochs):
    train(model, optimizer, r_train_dataloader, criterion, device)
    auc_score = test(model, r_train_dataloader, device)
    print('epoch:', epoch, 'validation: roc_auc_score:', auc_score)

100%|██████████| 53/53 [00:00<00:00, 523.17it/s]


epoch: 0 validation: roc_auc_score: 0.5627054832791694


100%|██████████| 53/53 [00:00<00:00, 467.24it/s]


epoch: 1 validation: roc_auc_score: 0.8867796508536988


100%|██████████| 53/53 [00:00<00:00, 509.78it/s]


...

epoch: 97 validation: roc_auc_score: 0.9999986535934438


100%|██████████| 53/53 [00:00<00:00, 535.46it/s]


epoch: 98 validation: roc_auc_score: 0.9999998316991805


100%|██████████| 53/53 [00:00<00:00, 578.72it/s]

epoch: 99 validation: roc_auc_score: 0.9999998076562062


In [ ]:
test(model, r_test_dataloader, device)

100%|██████████| 14/14 [00:00<00:00, 560.21it/s]


0.7172484886275323

## DCN

In [ ]:
class CrossNetwork(nn.Module):

    def __init__(self, input_dim: int, num_layers: int):
        super().__init__()
        self.num_layers = num_layers
        self.w = torch.nn.ModuleList([
            torch.nn.Linear(input_dim, 1, bias=False) for _ in range(num_layers)
        ])
        self.b = torch.nn.ParameterList([
            torch.nn.Parameter(torch.zeros((input_dim,))) for _ in range(num_layers)
        ])

    def forward(self, x: torch.Tensor):
        """
        :param x: Float tensor of size ``(batch_size, num_fields, embed_dim)``
        """
        x0 = x
        for i in range(self.num_layers):
            xw = self.w[i](x)
            x = x0 * xw + self.b[i] + x
        return x

Stacked DCN 모델 구조입니다

In [ ]:
class StackedDeepCrossNetworkModel(nn.Module):
  
    def __init__(self, field_dims: np.ndarray, embed_dim: int, num_layers: int, mlp_dims: tuple, dropout: float):
        super().__init__()
        self.embedding = FeaturesEmbedding(field_dims, embed_dim)
        self.embed_output_dim = len(field_dims) * embed_dim
        self.cn = CrossNetwork(self.embed_output_dim, num_layers)
        self.mlp = MultiLayerPerceptron(self.embed_output_dim, mlp_dims, dropout, output_layer=False)
        self.linear = torch.nn.Linear(self.embed_output_dim, 1, bias=False)

    def forward(self, x: torch.Tensor):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        embed_x = self.embedding(x).view(-1, self.embed_output_dim)
        x_l1 = self.cn(embed_x)
        x_out = self.mlp(x_l1)
        p = self.linear(x_out)
        return torch.sigmoid(p.squeeze(1))

Parallel DCN 구조입니다

In [ ]:
class ParallelDeepCrossNetworkModel(nn.Module):

    def __init__(self, field_dims: np.ndarray, embed_dim: int, num_layers: int, mlp_dims: tuple, dropout: float):
        super().__init__()
        self.embedding = FeaturesEmbedding(field_dims, embed_dim)
        self.embed_output_dim = len(field_dims) * embed_dim
        self.cn = CrossNetwork(self.embed_output_dim, num_layers)
        self.mlp = MultiLayerPerceptron(self.embed_output_dim, mlp_dims, dropout, output_layer=False)
        self.linear = torch.nn.Linear(mlp_dims[-1] + self.embed_output_dim, 1)

    def forward(self, x):
        """
        :param x: Long tensor of size ``(batch_size, num_fields)``
        """
        embed_x = self.embedding(x).view(-1, self.embed_output_dim)
        x_l1 = self.cn(embed_x)
        h_l2 = self.mlp(embed_x)
        x_stack = torch.cat([x_l1, h_l2], dim=1)
        p = self.linear(x_stack)
        return torch.sigmoid(p.squeeze(1))

In [ ]:
field_dims = np.array([len(user2idx), len(book2idx)], dtype=np.uint32)

In [ ]:
criterion = torch.nn.BCELoss()
model = StackedDeepCrossNetworkModel(field_dims, embed_dim, num_layers=3, mlp_dims=(16, 16), dropout=0.2).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate, amsgrad=True, weight_decay=weight_decay)

for epoch in range(epochs):
    train(model, optimizer, r_train_dataloader, criterion, device)
    auc_score = test(model, r_train_dataloader, device)
    print('epoch:', epoch, 'validation: roc_auc_score:', auc_score)

100%|██████████| 53/53 [00:00<00:00, 489.58it/s]


epoch: 0 validation: roc_auc_score: 0.7537203737663069


100%|██████████| 53/53 [00:00<00:00, 466.85it/s]


epoch: 1 validation: roc_auc_score: 0.8838325472242482


100%|██████████| 53/53 [00:00<00:00, 447.68it/s]


...

epoch: 97 validation: roc_auc_score: 0.9996545745893844


100%|██████████| 53/53 [00:00<00:00, 473.01it/s]


epoch: 98 validation: roc_auc_score: 0.9999129884762987


100%|██████████| 53/53 [00:00<00:00, 469.64it/s]

epoch: 99 validation: roc_auc_score: 0.9996495736507468


In [ ]:
test(model, r_test_dataloader, device)

100%|██████████| 14/14 [00:00<00:00, 458.59it/s]


0.7146135661428352

<font color='red'><b>**WARNING**</b></font> : **본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다.** 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다.

